In [ ]:
import numpy as np
import distance_measures as measures
from typing import Callable

class DBSCAN:
    """
    Density-Based Spatial Clustering of Applications with Noise (DBSCAN) implementation.
    """

    def __init__(self, min_pts: int = 3, dist_meas: Callable = measures.Euclidean_distance, eps: float = 0.5) -> None:
        """
        Initializes the DBSCAN clustering algorithm.

        Args:
            min_pts (int): Minimum number of points to form a dense region (cluster).
            dist_meas (Callable): Distance measure function.
            eps (float): Maximum distance between two samples for them to be considered as in the same neighborhood.
        """
        self.min_pts: int = min_pts
        self.dist_meas: Callable = dist_meas
        self.eps: float = eps

    def form_cluster(self, X: np.ndarray, P: np.ndarray, P_id: int) -> np.ndarray:
        """
        Forms a cluster around point P.

        Args:
            X (np.ndarray): Dataset of shape (n_samples, n_features).
            P (np.ndarray): The point around which to form the cluster.
            P_id (int): Index of point P in X.

        Returns:
            np.ndarray: Binary array indicating cluster membership (1 if in cluster, 0 otherwise).
        """
        # Find indices of points not yet assigned to any cluster
        unclustered = np.where(self.cluster == 0)[0]

        # Compute distances from P only for unclustered points
        distances = np.apply_along_axis(lambda x: self.dist_meas(x, P), 1, X[unclustered])

        # Find neighbors within eps distance
        neighbours_ids = unclustered[distances < self.eps]

        # Note: P is its own neighbor (distance 0 < eps)
        # To avoid infinite recursion, do not pass P as a neighbor in recursive calls

        if neighbours_ids.shape[0] >= self.min_pts:
            # Enough neighbors to form a cluster
            for idx, i in enumerate(neighbours_ids):
                # Only expand to neighbors that are not P itself

                if distances[idx] > 0:
                    # Recursively form clusters for neighbors
                    self.form_cluster(X, X[i, :], i)  

                    # TO DO
                    # Find all the points belonging to subclusters and merge them into one cluster
                    

            # Optionally, assign cluster label here if needed
        else:
            # Not enough neighbors, return zeros (noise)
            return np.zeros(X.shape[0], dtype=int)

    def fit(self, X: np.ndarray) -> np.ndarray:
        """
        Fits the DBSCAN clustering algorithm to the data.

        Args:
            X (np.ndarray): Dataset of shape (n_samples, n_features).

        Returns:
            np.ndarray: Array of cluster assignments for each sample.
        """
        n: int = X.shape[0]  # Number of samples
        self.cluster: np.ndarray = np.zeros(n, np.int8)  # 0 means noise/unassigned

        for i in range(n):
            if self.cluster[i] == 0:
                P: np.ndarray = X[i, :]  # Starting point for cluster
                self.form_cluster(X, P, i)  # Form cluster around point P

        return self.cluster